In [1]:
import pandas as pd 
import os 
import GEOparse
import scanpy as sc

debug = True 

## Barcode Harmonization Functions

#### This barcode harmonization code includes: automated metadata and data downloads from GEO database, cell barcode uniqueness.  This works for any multiomic single cell data. This is applied to datasets from the CITA-CD4 atlas.    

In [ ]:
def generate_metadata_file(geo, metadata_dir, include_tcr = False): 
            
    filepath = None
    
    geostr= geo 
    if geo == 'GSE173351':
        filepath = metadata_dir + geo + '_family.soft.gz'
        geostr = None

    print(filepath)
    gse = GEOparse.get_GEO(geo=geostr, filepath = filepath, destdir=metadata_dir)
    
    metadata = pd.DataFrame()
    
    for gsm_name, gsm in gse.gsms.items():
        tmp = pd.DataFrame()
        keys = []
        values = [] 
        for key, value in gsm.metadata.items():
            keys.append(str(key))
            values.append(str(value[0]))

        tmp['value'] = values
        tmp = tmp.transpose()
        
        print(tmp)
        if 'TCR' in ''.join(values):
            if include_tcr: 
                metadata = pd.concat([metadata,tmp])
        else: 
            metadata = pd.concat([metadata,tmp])

    metadata = metadata.rename(columns=dict(zip(metadata.columns, keys))).reset_index().drop('index', axis=1)
    
    if 'geo_accession' not in metadata.columns:
        geo_acc = 'Series_geo_accession'
    metadata = metadata.set_index('geo_accession')
    metadata.to_csv(metadata_dir + geo + '_geo_metadata.csv')
    print('Output GEO metadata to ' + metadata_dir + geo + '_geo_metadata.csv')

    return metadata

In [ ]:
def read_metadata_file(geo, metadata_dir):
    
    metadata = pd.read_csv(metadata_dir + geo + '_geo_metadata.csv')
    print('Reading GEO metadata to ' + metadata_dir + geo + '_geo_metadata.csv')
    return metadata 
    

In [ ]:
from ftplib import FTP
from more_itertools import locate

def download_data(download_dir, download=False, untar=False):
    status = 0
    if download:
        make_directory(gex_dir, geo +'_RAW')
        for gsm in metadata.index: 
            cmd = metadata.loc[gsm].supplementary_file_1
            if 'ftp' not in cmd: 
                print(' Error no ftp command in metadata.supplementary_file_1')
            else:
                status = download_data(gex_dir + geo +'_RAW/', cmd)              
                ftp = FTP('ftp.ncbi.nlm.nih.gov')
                ftp.login()    
                indlist  = list(locate(list(cmd), lambda a: a == '/'))
                
                file = cmd[indlist[-1]+1:]
                path = cmd[indlist[2]:indlist[-1]+1]
        
                print('Downloading ', path + file)
                print('Download directory ', download_dir)
    
                ftp.cwd(path)
                status = ftp.retrbinary("RETR " + file ,open(download_dir + file  , 'wb').write)
                ftp.close()
    
    if untar:
        untar_data(gex_dir, geo +'_RAW/')

    return status 


In [ ]:
def read_barcodes(path_to_barcode, barcode_file): 
    
    if debug:
        print('read_barcodes', path_to_barcode, barcode_file)

    if 'barcodes' in barcode_file:
        df = pd.read_table(path_to_barcode +  barcode_file , header=None) 
    elif  'csv' in barcode_file:
        gsm = barcode_file[0:10]
        df = pd.DataFrame(pd.read_csv(path_to_barcode +  barcode_file).iloc[:,0])
        df = df.rename(columns={'Unnamed: 0':'Original Barcode'})
        df['Original Barcode'] = gsm + '_' + df['Original Barcode'].astype(str)
        
    elif  'txt' in barcode_file:

        gsm = barcode_file[0:10]
        
        df = pd.DataFrame(pd.read_table(path_to_barcode +  barcode_file).iloc[:,0])

    elif 'h5' in barcode_file:
        df = sc.read_10x_h5(path_to_barcode + barcode_file)
  
    return df 

In [ ]:
def harmonize_barcodes_no_subdir(geo, download_dir, subdir, output_dir, metadata): 
    
    alldata = pd.DataFrame()  

    for root, subdirs, files in os.walk(download_dir + subdir):  
        
        print(subdirs)
   
        for barcode_file in files:  
         
            if 'barcode' in barcode_file or 'count_matrix' in barcode_file: 
                print(download_dir + subdir , barcode_file)
                df = read_barcodes(download_dir + subdir + '/', barcode_file)
                    
                sample_geo= barcode_file[0:10]  
                metadata_ss = metadata.loc[sample_geo]
                                         
                sample_title = metadata_ss.title
            
                df['Sample GEO'] = sample_geo
                df['Series GEO'] = geo
                df['Sample Title'] = sample_title
                df = df.rename(columns={0:'Original Barcode'})
                alldata= pd.concat([alldata, df])   
            
    alldata = output_barcodes(alldata, output_dir, geo)
    return alldata

In [ ]:
def harmonize_barcodes(geo, download_dir, subdir, output_dir, metadata): 
    
    alldata = pd.DataFrame()  

    for root, subdirs, files in os.walk(download_dir + subdir): 
        
      
        for s in subdirs:
            for root2, subdirs2, files2 in os.walk(download_dir + subdir +  s +'/'):  
                for barcode_file in files2:                
                    if 'barcode' in barcode_file:  
                        if 'tar.gz' in barcode_file: 
                            cmd1 = 'tar -xvzf ' + download_dir + subdir +  s +'/' + f
                            os.system(cmd)
                            barcode_file = barcode_file[:-7]
                            print(barcode_file)
                            break
                            
                        df = read_barcodes(download_dir + subdir +  s +'/', barcode_file)
                
                        sample_geo= s[0:10]       
                        metadata_ss = metadata.loc[sample_geo]
                        sample_title = metadata_ss.title
    
                        df['Sample GEO'] = sample_geo
                        df['Series GEO'] = geo
                        df['Sample Title'] = sample_title
                        df = df.rename(columns={0:'Original Barcode'})
                        alldata= pd.concat([alldata, df])       
                        
    alldata = output_barcodes(alldata, output_dir, geo)
    return alldata 

In [ ]:
def make_directory(path, folder):
    
    exists = os.path.exists(path+folder)
    if ~exists:
        cmd = 'mkdir ' + path + folder
        os.system(cmd)
    

In [ ]:
def untar_data(path, folder):
    for roots, subdir, files in os.walk(path + folder): 
        for f in files: 
            if 'GSM' in f and 'tar.gz' in f: 
                cmd1 = 'tar -xvzf ' + path + folder + f 
            elif 'GSM' in f or 'GSE' in f and '.gz' in f: 
                cmd1 = 'gunzip ' + path + folder + f 
            elif 'GSM' in f and '.tar' in f: 
                cmd1 = 'tar -xvf ' + path + folder + f 
            print(cmd1)
            os.system(cmd1)          
    for roots, subdir, files in os.walk(path + folder): 
        for f in files: 
            print(path +folder +f)

In [ ]:
def reduce_files(path, folder, top=True, num_lines=5):
    
    for root, subdirs, files in os.walk(path, folder): 
        for f in files: 
            status += reduce_file(path, folder, filename=f, top=True, num_lines=5)
    return status

In [ ]:
def reduce_file(path, folder, filename, top=True, num_lines=5):
 
    suffix = 'txt'
    if 'csv' in filename:
        suffix = 'csv'
    elif 'tsv' in f: 
        suffix = 'tsv'
    cmd = 'head -n ' + str(num_lines) + ' ' + path + folder + filename + ' > ' + path + folder + filename +'.small.'+ suffix
    
    print(cmd)
    status = os.system(cmd)
    return status

In [ ]:
def output_barcodes(alldata, output_dir, geo, barcode_type='gex'):
   
    alldata['Series GEO'] = geo 
    
    if 'Sample GEO' in alldata.columns: 
        alldata['Barcode'] = alldata['Original Barcode'] + '_' + alldata['Sample GEO']
    else: 
        alldata['Sample GEO'] = ''
        alldata['Barcode'] = alldata['Original Barcode'] + '_' + alldata['Series GEO']
    
    alldata = alldata.drop_duplicates()
    print('Original Barcode/Unique Original Barcode', len(alldata['Original Barcode']), len(alldata['Original Barcode'].unique()))
    print('Barcode/Unique Barcode', len(alldata['Barcode']), len(alldata['Barcode'].unique()))

    columns = ['Barcode', 'Original Barcode', 'Series GEO', 'Sample GEO']
    alldata = alldata[columns]
    
    alldata.to_csv(output_dir + geo + '_barcode_unique.csv', index=None)
    print('Output barcodes to ' + output_dir + geo + '_barcode_unique_' + barcode_type + '.csv')
 
    return alldata

In [ ]:
def read_geos_file(path, filename):   
    return pd.read_csv(path+filename, header=None).iloc[:,0].values

## Harmonize barcodes for CITA-CD4 Atlas 

In [ ]:
gex_dir = '/Users/vanessajonsson/Google_Drive/data/rep/cd4_atlas/data/gex/'
metadata_dir = '/Users/vanessajonsson/Google_Drive/data/rep/cd4_atlas/data/metadata/'
output_dir = '../output/barcode/'

geos = read_geos_file(metadata_dir, filename='geolist.csv')

geo = geos[2]
subdir = geo +'_RAW/'

download = False 
untar = False 
    
metadata = generate_metadata_file(geo, metadata_dir, include_tcr = False)
download_data(gex_dir + subdir, download=download, untar = untar)

geo 

In [ ]:
if geo == 'GSE173351':
    
    alldata = pd.DataFrame()  

    for root, subdirs, files in os.walk(gex_dir + subdir): 
        
        for s in subdirs:
            for root2, subdirs2, files2 in os.walk(gex_dir + subdir +  s +'/'):  
                for barcode_file in files2:                
                    if 'barcode' in barcode_file:  
                        if 'tar.gz' in barcode_file: 
                            cmd1 = 'tar -xvzf ' + gex_dir + subdir +  s +'/' + f
                            os.system(cmd)
                            barcode_file = barcode_file[:-7]
                            
                            
                        df = read_barcodes(gex_dir + subdir +  s +'/', barcode_file)
                        dd = dict(zip(metadata.title, metadata.index))
                        sample_geo= dd[s]       
                        metadata_ss = metadata.loc[sample_geo]
    
                        df['Sample GEO'] = sample_geo
                        df['Series GEO'] = geo
                        df = df.rename(columns={0:'Original Barcode'})
                        alldata= pd.concat([alldata, df])       
                        
    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE114724' or geo == 'GSE139555':
    
    alldata = pd.DataFrame()
    for root, dirs, files in os.walk( gex_dir + subdir ):
        
        for f in files:
            df= pd.DataFrame()
            df = pd.DataFrame(pd.read_table(gex_dir + subdir + f , header=None))
            df['Sample GEO'] = f[0:10]
            df = df.drop_duplicates()
            alldata = pd.concat([alldata, df])
            del df 
    
    alldata['Original Barcode'] = alldata.iloc[:,0].astype(str)
    output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE156728':
    
    alldata = pd.DataFrame()
    for root, dirs, files in os.walk( gex_dir + subdir ):
        
        for f in files:
            print(f)
            if '.DS_store' not in f:
                df= pd.DataFrame()
                df = pd.DataFrame(pd.read_table(gex_dir + subdir + f, nrows=1).columns[1:])
                df['Sample GEO'] = f[0:10]
                df = df.drop_duplicates()
                alldata = pd.concat([alldata, df])
                del df 
    
    alldata['Original Barcode'] = alldata.iloc[:,0].astype(str)
    output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE123139' or geo =='GSE139555':
    
    alldata = pd.DataFrame()
    for root, dirs, files in os.walk( gex_dir + subdir ):
        for f in files: 
            df= pd.DataFrame()
            df = pd.DataFrame(pd.read_table(gex_dir + subdir + f).columns)
            df['Sample GEO'] = f[0:10]
            df = df.drop_duplicates()
            alldata = pd.concat([alldata, df])
            del df 
    
    alldata['Original Barcode'] = alldata.iloc[:,0].astype(str)
    output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE148190' or geo == 'GSE123902' :
    
    alldata = pd.DataFrame()
    for root, dirs, files in os.walk( gex_dir + subdir ):
        for f in files: 
            print(f)
            df= pd.DataFrame()
            df = pd.DataFrame(pd.read_csv(gex_dir + subdir + f).iloc[:,0])
            df['Sample GEO'] = f[0:10]
            
            alldata = pd.concat([alldata, df])
            del df 
    
    alldata['Original Barcode'] = alldata.iloc[:,0].astype(str)
    
    output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE146771':
    
    f='GSE146771_CRC.Leukocyte.10x.Metadata.txt'

    metadata['Donor'] = metadata.characteristics_ch1.str[9:]
    dict_donor = dict(zip(metadata.Donor, metadata.index))

    df = pd.DataFrame(pd.read_table(gex_dir + subdir + f))[['Sample_BarcodeID']].rename(columns={'Sample_BarcodeID':'Original Barcode'})
    df['Donor'] = df['Original Barcode'].str.split('_',expand=True).iloc[:,1]
    df['Sample GEO'] = [dict_donor[donor] for donor in df.Donor]

    output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE176078' or geo == 'GSE145370':
    
    alldata = harmonize_barcodes(geo, gex_dir, subdir, output_dir, metadata)
    output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE99254':

    f='GSE99254_NSCLC.TCell.S12346.count.txt'

    df = pd.DataFrame(pd.read_table(gex_dir + subdir + f, nrows=1).columns[2:]).rename(columns={0:'Original Barcode'})
    alldata = output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE130148':
    
    f ='GSE130148_barcodes_cell_types.txt'

    df = pd.DataFrame(pd.read_table(gex_dir + subdir + f))[['cell.barcode']].rename(columns={'cell.barcode':'Original Barcode'})
    alldata = output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE108989':
    
    f ='GSE108989_CRC.TCell.S11138.count.txt'

    df = pd.DataFrame(pd.read_table(gex_dir + subdir + f, nrows = 2).columns[2:]).rename(columns={0:'Original Barcode'})
    alldata = output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE72056':
    
    f ='GSE72056_melanoma_single_cell_revised_v2.txt'

    dd = dict(zip(metadata.title, metadata.index))

    df = pd.DataFrame(pd.read_table(gex_dir + subdir + f, nrows = 2).columns[1:]).rename(columns={0:'Original Barcode'}) 
    df['Sample GEO'] = [ dd[bc] for bc in df['Original Barcode']] 

    alldata = output_barcodes(df, output_dir, geo)

In [ ]:
if geo == 'GSE162025':

    alldata = pd.DataFrame()
    for root, subdirs, files in os.walk(gex_dir + subdir): 
        for f in files: 
            print(f[0:10])
            df = pd.DataFrame(pd.read_csv(gex_dir + subdir + f).columns).rename(columns={0:'Original Barcode'})
            df = df.drop_duplicates()
            df = df.dropna()
            df['Sample GEO'] = f[0:10]
            alldata = pd.concat([alldata, df])

    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE127465':

    for root, subdirs, files in os.walk(gex_dir + subdir):
        for f in files: 
            if '.DS_Store' not in f: 
                data = pd.DataFrame(pd.read_table(gex_dir + subdir  +  f, usecols=['barcode']))
                data = data.rename(columns={'barcode':'Original Barcode'})
                
                data['Sample GEO'] = f[0:10]
                data = data.dropna(axis=0)

                alldata = pd.concat([alldata, data])

    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE182434':
    
    f='GSE182434_raw_count_matrix.txt.small.csv.small.txt'

    alldata = pd.DataFrame(pd.read_table(gex_dir + subdir  + f).columns[1:])
    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})
    alldata['Sample GEO'] = '' 
    alldata = alldata.drop_duplicates().dropna() 

    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE120575':
    
    f = 'GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt'
    s ='GSE120575_patient_ID_single_cells.txt'
    
    samples = pd.DataFrame(pd.read_table(gex_dir + subdir  +  s)).dropna(axis=1)

    dd = dict(zip(samples['Unnamed: 1'], samples['Unnamed: 4']))
    ddmeta = dict(zip(metadata.title, metadata.index))

    alldata = pd.DataFrame(pd.read_table(gex_dir + subdir  +  f, nrows=1).columns[1:])
    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})
    alldata['Sample GEO'] = [ddmeta[dd[bc]] for bc in alldata['Original Barcode']] 
    alldata = alldata.drop_duplicates().dropna() 
    
    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE131907':

    f= 'GSE131907_Lung_Cancer_raw_UMI_matrix.txt.small.txt'

    alldata = pd.DataFrame(pd.read_table(gex_dir + subdir  + f).columns[1:])
    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})

    dd = dict(zip(metadata.title, metadata.index))

    alldata['Sample'] = alldata['Original Barcode'].str[17:]
    alldata['Sample GEO'] = [dd[sam] for sam in alldata.Sample]
    alldata = output_barcodes(alldata, output_dir, geo)


In [ ]:
if geo == 'GSE98638':
  
    f='GSE98638_HCC.TCell.S5063.count.txt.small.txt'

    alldata = pd.DataFrame(pd.read_table(gex_dir + subdir + f).columns[2:])
    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})
    dd = dict(zip(metadata.title, metadata.index))

    alldata = output_barcodes(alldata, output_dir, geo)


In [ ]:
if geo == 'GSE203183':

    f = 'GSE203183_Schad_2_Gene_counts.txt.small.txt'

    alldata = pd.DataFrame(pd.read_table(gex_dir + subdir  +  f, sep=','))
    alldata = pd.DataFrame(alldata.columns[1:])
    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})
    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE125449':

    bc_files = ['GSE125449_Set1_barcodes.tsv', 'GSE125449_Set2_barcodes.tsv' ] 
    sm_files = ['GSE125449_Set1_samples.txt','GSE125449_Set2_samples.txt']

    dd = dict(zip(bc_files, sm_files))
    meta_dict = dict(zip(metadata.title, metadata.index))

    alldata = pd.DataFrame() 
    for f in bc_files: 
        df  = pd.DataFrame(pd.read_table(gex_dir + subdir  +  f , sep='\t', header=None))
        sm  = pd.DataFrame(pd.read_table(gex_dir + subdir  +  dd[f]))
        data = df.merge(sm, how='inner', left_on=0, right_on='Cell Barcode')
        data['Sample GEO'] = [meta_dict[sam] for sam in data.Sample]
        alldata= pd.concat([alldata, data])

    alldata = alldata.reset_index().rename(columns={0:'Original Barcode'})
    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE163108':
    alldata = pd.DataFrame()
    for root, dirs, files in os.walk(gex_dir + subdir):
        for f in files: 
            df = sc.read_10x_h5(gex_dir + subdir + f).obs
            df['Sample GEO'] = f[0:10]
            alldata= pd.concat([alldata, df])
            

    alldata = alldata.reset_index().rename(columns={'index':'Original Barcode'})
    alldata = output_barcodes(alldata, output_dir, geo)

In [ ]:
if geo == 'GSE121636':
    alldata = harmonize_barcodes_no_subdir(geo, gex_dir, subdir, output_dir, metadata)

In [ ]:
if geo == 'GSE115978':
    
    f ='GSE115978_counts.csv'
    
    alldata = pd.DataFrame(pd.read_csv(gex_dir + subdir + f, nrows=1).columns[1:])

    alldata = alldata.rename(columns = {0:'Original Barcode'})
    alldata = output_barcodes(alldata, output_dir, geo)

    sample_geos = [] 
    for bc in alldata['Original Barcode']: 
        sample_geos.append(metadata.loc[metadata.title == bc].index[0] ) 
     
    alldata['Sample GEO'] = sample_geos 
    alldata = output_barcodes(alldata, output_dir, geo)
   

In [ ]:
if geo == 'GSE123813':

    df1 = pd.DataFrame(pd.read_table(gex_dir +'GSE123813_RAW/GSE123813_scc_scRNA_counts.small.txt').columns)
    df2 = pd.DataFrame(pd.read_table(gex_dir +'GSE123813_RAW/GSE123813_bcc_scRNA_counts.small.txt').columns)

    alldata = pd.concat([df1, df2])

    alldata = alldata.rename(columns = {0:'Original Barcode'})
    alldata = output_barcodes(alldata, output_dir, geo)